In [1]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
import json
import time

In [2]:

load_dotenv()

url = "https://api.newsedge.moodysanalytics.com/news"
headers = {
    "Authorization": os.getenv("M_TOKEN"),
    "accept": "application/json"
}

params = {
    "facets_stats_top": 1,
    "page_size": 25,
    "page_number": 1,
    "enable_stemming": "false",
    "enable_highlighting": "false",
    "query" : "Zomato"
}
r = requests.get(url=url, headers=headers, params=params)

print()
print(r)



<Response [200]>


In [9]:
df = pd.read_csv("./data/non us moody mapping - total not found.csv")

def cleaner(s):
    s = s.lower().replace(',', '').replace('.', '').replace("'", '').replace(' inc', '').replace(' corporation', '')
    return '"' + s.replace(' co', '').strip() + '"'

df['cleaned'] = df['db_name'].apply(cleaner)

print(df)
    

     db_ticker                       db_name                      cleaned
0      TCLA_CA              Transcontinental           "transcontinental"
1          CLB        Core Laboratories N.V.       "core laboratories nv"
2          AGL           agilon health, inc.              "agilon health"
3      MTUS_US                      Metallus                   "metallus"
4         SPWR          SunPower Corporation                   "sunpower"
..         ...                           ...                          ...
757  000725_CN  BOE Technology Group Company  "boe technology groupmpany"
758  000651_CN      Gree Electric Appliances   "gree electric appliances"
759  000338_CN                 Weichai Power              "weichai power"
760  000425_CN   XCMG Construction Machinery   "xcmgnstruction machinery"
761  000063_CN                           ZTE                        "zte"

[762 rows x 3 columns]


In [10]:
data = {'ticker':[], 'db_name':[],'cleaned':[], 'name':[],'orbis_id':[], 'id':[], 'symbol':[] }
no_story = []
# print(json.dumps(r.json()['data']['stories'][0]['organizations'], indent=2))

for ticker, db_name, name in zip(df['db_ticker'], df['db_name'], df['cleaned']):
    params["query"] = name
    r = requests.get(url=url, headers=headers, params=params)
    
    print(ticker, name)
    
    print(r.status_code)
    if(r.status_code != 200):
        print(r.reason, r.text)
        print(ticker)
        print()
        break
    try:
        orgs = r.json()['data']['stories'][0]['organizations']
    except (KeyError, IndexError) as e:
        print('e1', e, name)
        no_story.append((ticker, name))
        continue
        


    # print(json.dumps(orgs, indent=2))
    for data_value in orgs:
        try:
            data['cleaned'].append(name)
            data['ticker'].append(ticker)
            data['db_name'].append(db_name)
            data['name'].append(data_value['name'])
            data["orbis_id"].append(data_value['orbis_id'])
            data["id"].append(data_value['id'])
            try:
                data["symbol"].append(data_value['securities'][0]['symbol'])
            except IndexError:
                data["symbol"].append("")
        except (KeyError, IndexError) as e:
            print('e2', e, data_value)
    time.sleep(0.1)
    
print(*data, sep='\n')
for k, v in data.items():
    print(k, len(v))

TCLA_CA "transcontinental"
200
CLB "core laboratories nv"
200
AGL "agilon health"
200
MTUS_US "metallus"
200
SPWR "sunpower"
200
REGI_US "renewable energy group"
200
AI "c3ai"
200
QFIN "360 digitech"
200
GIB "cgi"
200
GCI "gannett"
200
INSE "inspired entertainment"
200
JD "jdcom"
200
PINE "alpineome property trust"
200
e1 list index out of range "alpineome property trust"
AGI "alamos gold"
200
GE "general electricmpany"
200
e1 list index out of range "general electricmpany"
BATRK_US "atlanta braves holdings"
200
ALV "autoliv"
200
MBC.W_US "masterbrand"
200
FDP_US "fresh del monte produce"
200
TRS_US "trimas"
200
MAC "the macerichmpany"
200
e1 list index out of range "the macerichmpany"
UVSP "univest of pennsylvania"
200
e1 list index out of range "univest of pennsylvania"
OFIX "orthofix international nv"
200
e1 list index out of range "orthofix international nv"
ISPO "inspiratoorporated"
200
e1 list index out of range "inspiratoorporated"
IMUX "immunic therapeutics"
200
GCO "genesco"
2

In [12]:
df = pd.DataFrame(data)
# for k, v in data.items():
#     print(k, len(v))
# df['orbis id'], df['id'], df['symbol'] = data['orbis_id'], data['id'], data['symbol']
df.to_csv(r'.\data\extracted v3.csv')



In [51]:
import os
print(os.getcwd())

C:\Users\Divya\PycharmProjects\VQ Scraper
